In [1]:
import os
import google.generativeai as genai

os.environ["GOOGLE_API_KEY"] = "MY_API_KEY"
# Set your Gemini API key
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Optional: choose a stable model
model = genai.GenerativeModel("gemini-robotics-er-1.5-preview")


In [2]:
def get_completion_from_messages(messages,
                                 model="models/gemini-robotics-er-1.5-preview",
                                 temperature=0,
                                 max_tokens=1500):   # <-- FIXED
    """
    Same behavior as your OpenAI function,
    but using Gemini + enough tokens for your formatted output.
    """

    model_instance = genai.GenerativeModel(model)

    # Build prompt in the same format you used
    prompt = ""
    for m in messages:
        role = m["role"].capitalize()
        prompt += f"{role}: {m['content']}\n"

    # Call Gemini
    response = model_instance.generate_content(
        prompt,
        generation_config={
            "temperature": temperature,
            "max_output_tokens": max_tokens
        }
    )

    # Safe extractor (Gemini sometimes doesn't support .text directly)
    try:
        return response.text
    except:
        if response.candidates:
            cand = response.candidates[0]
            if hasattr(cand, "content") and cand.content.parts:
                return "".join([p.text for p in cand.content.parts if hasattr(p, "text")])
        return "ERROR: Gemini returned no usable text."

Chain of thought Reasoning

In [3]:
delimiter = "####"

system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags, i.e. {delimiter}.

Step 1:{delimiter} First decide whether the user is asking a question about a specific product or products. Product category doesn't count.

Step 2:{delimiter} If the user is asking about specific products, identify whether the products are in the following list.
All available products:
1. Product: TechPro Ultrabook
   Price: $799.99
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5
   Description: A sleek and lightweight ultrabook.

2. Product: BlueWave Gaming Laptop
   Price: $1199.99
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA RTX 3060
   Description: A high-performance gaming laptop.

3. Product: PowerLite Convertible
   Price: $699.99
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop.

4. Product: TechPro Desktop
   Price: $999.99
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7, 16GB RAM, 1TB HDD, NVIDIA GTX 1660
   Description: A powerful desktop computer.

5. Product: BlueWave Chromebook
   Price: $249.99
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook.

Step 3:{delimiter} If the message contains products in the list above, list any assumptions that the user is making (e.g. that Product X is bigger than Product Y, or that Product Z has a 2 year warranty).

Step 4:{delimiter} If the user made any assumptions, determine whether each assumption is true based on the product information above.

Step 5:{delimiter} Politely correct any incorrect assumptions. Only mention the five products above (these are the only products the store sells). Answer in a friendly tone.

Output format (use the delimiter to separate every step exactly as shown):
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>
"""



In [4]:
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": f"{delimiter}{user_message}{delimiter}"}
]

# --------- Call the Gemini wrapper ----------
try:
    response = get_completion_from_messages(messages)
    print("=== MODEL RESPONSE ===\n")
    print(response)
except RuntimeError as e:
    print("Model call failed:\n", e)

=== MODEL RESPONSE ===

Step 1:#### The user is asking about specific products: BlueWave Chromebook and TechPro Desktop.
Step 2:#### Both products (BlueWave Chromebook and TechPro Desktop) are in the available product list.
Step 3:#### The user assumes that the BlueWave Chromebook is more expensive than the TechPro Desktop.
Step 4:#### The assumption is incorrect. The BlueWave Chromebook costs $249.99, while the TechPro Desktop costs $999.99. The TechPro Desktop is more expensive.
Response to user:#### Hi there! It looks like you might have a small assumption about the prices. The TechPro Desktop is actually more expensive than the BlueWave Chromebook. The TechPro Desktop costs $999.99, and the BlueWave Chromebook costs $249.99, making the TechPro Desktop $750.00 more expensive.


In [5]:
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)

Hi there! It looks like you might have a small assumption about the prices. The TechPro Desktop is actually more expensive than the BlueWave Chromebook. The TechPro Desktop costs $999.99, and the BlueWave Chromebook costs $249.99, making the TechPro Desktop $750.00 more expensive.
